# setup environment

In [ ]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd
import torch

# Load the Model and Tokenizer

In [ ]:
# Use GPT-2 Medium
model_name = "gpt2-medium"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Assign padding token
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# Prepare Dataset

In [ ]:
# Load your dataset
data = pd.read_csv("cleaned_dataset.csv")
dataset = Dataset.from_pandas(data)

# Split the dataset into train and validation sets
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# Preprocessing function
def preprocess_function(examples):
    # Combine the client prompt and therapist response
    inputs = [f"Client: {prompt}\nTherapist: {response}" for prompt, response in zip(examples["client"], examples["therapist"])]
    tokenized = tokenizer(inputs, truncation=True, padding="max_length", max_length=128)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Tokenize datasets
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=["client", "therapist"])
eval_dataset = eval_dataset.map(preprocess_function, batched=True, remove_columns=["client", "therapist"])

Map:   0%|          | 0/574 [00:00<?, ? examples/s]

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

# Cell 4: tokenize dataset

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-medium-therapeutic",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,  # Adjust for memory constraints
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    save_strategy="epoch",
    save_total_limit=2,  # Save only the latest checkpoints
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    fp16=True,  # Enable mixed precision for GPU
    push_to_hub=False
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Define Training Arguments

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# Start training
trainer.train()

<ipython-input-12-0afd329bef55>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.157700,1.576889
2,0.908600,1.637157
3,0.774500,1.816243
4,0.539300,1.950435
5,0.412000,2.047741


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=1435, training_loss=0.7532982189896215, metrics={'train_runtime': 545.7449, 'train_samples_per_second': 5.259, 'train_steps_per_second': 2.629, 'total_flos': 666342748323840.0, 'train_loss': 0.7532982189896215, 'epoch': 5.0})

# train the model:

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./gpt2-medium-therapeutic")
tokenizer.save_pretrained("./gpt2-medium-therapeutic")

('./gpt2-medium-therapeutic/tokenizer_config.json',
 './gpt2-medium-therapeutic/special_tokens_map.json',
 './gpt2-medium-therapeutic/vocab.json',
 './gpt2-medium-therapeutic/merges.txt',
 './gpt2-medium-therapeutic/added_tokens.json',
 './gpt2-medium-therapeutic/tokenizer.json')

In [ ]:
# Save to Google Drive
from google.colab import drive
drive.mount('/content/drive')
model.save_pretrained('/content/drive/MyDrive/gpt2-medium-therapeutic')
tokenizer.save_pretrained('/content/drive/MyDrive/gpt2-medium-therapeutic')

Mounted at /content/drive


('/content/drive/MyDrive/gpt2-medium-therapeutic/tokenizer_config.json',
 '/content/drive/MyDrive/gpt2-medium-therapeutic/special_tokens_map.json',
 '/content/drive/MyDrive/gpt2-medium-therapeutic/vocab.json',
 '/content/drive/MyDrive/gpt2-medium-therapeutic/merges.txt',
 '/content/drive/MyDrive/gpt2-medium-therapeutic/added_tokens.json',
 '/content/drive/MyDrive/gpt2-medium-therapeutic/tokenizer.json')

In [ ]:
# Define the device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Function to generate responses
def generate_response(input_text):
    # Format the prompt
    prompt = f"Client: {input_text}\nTherapist:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=120,
        temperature=1,
        top_p=0.9,
        repetition_penalty=1.5,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )
    # Decode and clean the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "Therapist:" in response:
        response = response.split("Therapist:")[1].strip()
    response = response.split(".")[0].strip() + "."
    if not response.strip() or response == ".":
        response = "I'm here to listen and support you. You are not alone."
    return response

# save the finetuned model:

In [ ]:
# Test inputs
test_inputs = [
    "I'm feeling so overwhelmed at work. What should I do?",
    "I can't stop thinking about my breakup. It's consuming me.",
    "I feel like I’m not good enough for my family.",
    "I’ve lost someone I deeply care about, and I can’t move on."
]

# Generate responses
for input_text in test_inputs:
    response = generate_response(input_text)
    print(f"Input: {input_text}")
    print(f"Response: {response}\n")

Input: I'm feeling so overwhelmed at work. What should I do?
Response: There are a few things you can try to cope with this stress, even if it's not your own doing! One thing that may help is taking some time for yourself each day and focusing on something else entirely instead of dealing in numbers or deadlines all the way through school year again next semester!! Another option would be spending more quality sleep than normal because many people find they get tired during their lunch hour when there isn't much going around them either!!! Finally another possibility might involve finding ways where we don' t have too big an impact by simply being ourselves and having fun rather then worrying about how others.

Input: I can't stop thinking about my breakup. It's consuming me.
Response: There are a few things that you could do to try and get yourself out of this situation as quickly, painlessly or comfortably possible without causing too much stress for your partner in the long run.

In

In [ ]:
test_inputs = [
    "I've been feeling disconnected from my friends lately. What should I do?"
]

for input_text in test_inputs:
    response = generate_response(input_text)
    print(f"Input: {input_text}")
    print(f"Response: {response}\n")

Input: I've been feeling disconnected from my friends lately. What should I do?
Response: There are a few things you can try to reconnect with your social circle, even if it's just for one night or two weeks at most! One thing that may help is simply spending time alone in nature and doing something relaxing like yoga before going out into the world again so as not get overwhelmed by all of those people around us anymore!! Another way would be practicing some new activities on weekends where there isn't much activity for an extended period without getting too stressed about them causing problems.

